In [1]:
import numpy as np
import pandas as pd
import random
import keras
import os
import math
from decimal import *

from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D,ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import concatenate,BatchNormalization,Activation
from keras.utils import plot_model

import scipy.io
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.gridspec as gridspec
from datetime import datetime

from keras.layers import Conv2D, LeakyReLU
from keras.callbacks import EarlyStopping
from keras import regularizers
import tensorflow as tf
from keras.optimizers import SGD
import cv2 as cv, numpy as np

Using TensorFlow backend.


In [2]:
src = 'gd_aa_resize_128/'
src_hd = 'hd_aa_resize_128/'
save = 'save/'
files = os.listdir(src)
files_hd = os.listdir(src_hd)

In [3]:
def MAE(y_true, y_pred):     
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    return np.mean(np.abs((y_true - y_pred)))

count = 0
now = datetime.now() 
name =str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)

In [4]:
X = []
X_hd = []
for i in files:
    img = cv.imread(src+i,cv.COLOR_BGR2GRAY)
    X.append(img/255.)
    
for j in files_hd:
    img_hd = cv.imread(src_hd+j,cv.COLOR_BGR2GRAY)
    X_hd.append(img_hd/255.)

In [5]:
csv_train= pd.read_csv('gd_out_out.csv')
csv_test = pd.read_csv('hd_out_out.csv')
df_train = pd.DataFrame(csv_train)
df_test = pd.DataFrame(csv_test)

# 키확인
print(df_train.columns)

#두개의 열만 가져오기 
Y_temp_train = df_train[["freq_10","mass"]]
Y_temp_test = df_test[["freq_10","mass"]]

#행으로 자르기 
Y = Y_temp_train.iloc[:].values
Y_hd = Y_temp_test.iloc[:].values

X_train,Y_train = X,Y
X_test, Y_test = X_hd, Y_hd

print(np.shape(Y_test))

Index(['Unnamed: 0', 'name', 'volume', 'mass', 'freq_0', 'freq_1', 'freq_2',
       'freq_3', 'freq_4', 'freq_5', 'freq_6', 'freq_7', 'freq_8', 'freq_9',
       'freq_10', 'freq_11', 'freq_12', 'freq_13', 'freq_14', 'eig_val_0',
       'eig_val_1', 'eig_val_2', 'eig_val_3', 'eig_val_4', 'eig_val_5',
       'eig_val_6', 'eig_val_7', 'eig_val_8', 'eig_val_9', 'eig_val_10',
       'eig_val_11', 'eig_val_12', 'eig_val_13', 'eig_val_14', 'stiff_0',
       'stiff_1', 'stiff_2', 'stiff_3', 'stiff_4', 'stiff_5', 'stiff_6',
       'stiff_7', 'stiff_8', 'stiff_9', 'stiff_10', 'stiff_11', 'stiff_12',
       'stiff_13', 'stiff_14'],
      dtype='object')
(96, 2)


In [6]:
df_train

,Unnamed: 0,name,volume,mass,freq_0,freq_1,freq_2,freq_3,freq_4,freq_5,...,stiff_5,stiff_6,stiff_7,stiff_8,stiff_9,stiff_10,stiff_11,stiff_12,stiff_13,stiff_14
0,0,gd_0,4982300.0,0.013412,0.010401,0.010409,0.010799,0.011578,0.012236,0.012252,...,0.005926,4418217.0,4449240.0,20543880.0,21861350.0,21995310.0,55303530.0,55706230.0,59834680.0,61012500.0
1,1,gd_1,5452020.0,0.014677,0.010177,0.010236,0.010609,0.011387,0.012088,0.012163,...,0.005840,5071799.0,5096039.0,22722000.0,22732340.0,32827800.0,56334960.0,56377910.0,88099210.0,89578550.0
2,2,gd_2,6514530.0,0.017537,0.009940,0.009968,0.010305,0.011103,0.011820,0.011851,...,0.005545,6497199.0,6539628.0,24464300.0,24499260.0,54146880.0,57582400.0,57634940.0,108873300.0,109006200.0
3,3,gd_3,5462640.0,0.014705,0.010157,0.010217,0.010584,0.011384,0.012059,0.012115,...,0.005794,5247144.0,5273430.0,23065070.0,23201130.0,32070830.0,56564850.0,56844090.0,90970220.0,94296730.0
4,4,gd_4,5766630.0,0.015524,0.010091,0.010120,0.010501,0.011283,0.011999,0.012041,...,0.005724,5750682.0,5772577.0,23727910.0,23732980.0,39714390.0,57108480.0,57131510.0,108360800.0,108968900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,1001,gd_1025,5474840.0,0.014738,0.010018,0.010034,0.010467,0.011193,0.011988,0.012022,...,0.005706,5382878.0,5526187.0,23314140.0,23371450.0,36431730.0,56796570.0,56894630.0,100950600.0,108083800.0
1002,1002,gd_1026,6034180.0,0.016244,0.009980,0.010011,0.010380,0.011154,0.011869,0.011882,...,0.005574,6428272.0,6438410.0,25092470.0,25097240.0,45261190.0,58335560.0,58453020.0,109708400.0,109715100.0
1003,1003,gd_1027,5276620.0,0.014205,0.010335,0.010341,0.010717,0.011554,0.012210,0.012243,...,0.005918,4777378.0,4813749.0,21989620.0,22062490.0,31228520.0,55666380.0,55788670.0,94027600.0,94780990.0
1004,1004,gd_1028,5001540.0,0.013464,0.010389,0.010411,0.010793,0.011622,0.012274,0.012308,...,0.005980,4408315.0,4485384.0,21521900.0,21639450.0,24255620.0,50013750.0,54984390.0,55271840.0,57175210.0


In [7]:
X_train_shape = np.shape(X_train)
X_test_shape = np.shape(X_test)
print(X_train_shape,X_test_shape)

(1006, 128, 128) (96, 128, 128)


In [8]:
sc = MinMaxScaler()
Y_train_scale=sc.fit_transform(Y_train)
Y_test_scale=sc.fit_transform(Y_test)

In [9]:
X_train_reshape = np.reshape(X_train,(X_train_shape[0],X_train_shape[1],X_train_shape[2],1))
X_test_reshape = np.reshape(X_test,(X_test_shape[0],X_test_shape[1],X_test_shape[2],1))

print(np.shape(X_train_reshape),np.shape(X_test_reshape))
print(np.shape(Y_train_scale),np.shape(Y_test_scale))

(1006, 128, 128, 1) (96, 128, 128, 1)
(1006, 2) (96, 2)


In [10]:
leaky_relu = tf.nn.leaky_relu

def CNN(weights_path=None):

    model = Sequential()
    #model.add(ZeroPadding2D((1,0),input_shape=(225,1,1)))
    model.add(Conv2D(64, 3, 3, activation='relu',input_shape=(128,128,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(128, 3, 3,activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(256, 3, 3,activation='relu'))
    model.add(Conv2D(256, 3, 3,activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(512, 3, 3,activation='relu'))
    model.add(Conv2D(512, 3, 3,activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(512, 2, 2,activation='relu'))
    model.add(Conv2D(512, 2, 2,activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(2, activation='linear'))

    return model

def VGG19(weights_path=None):

    model = Sequential()
    #model.add(ZeroPadding2D((1,0),input_shape=(225,1,1)))
    model.add(Conv2D(64, 2, 2, activation='relu',input_shape=(128,128,1)))
    model.add(Conv2D(64, 2, 2,activation='relu'))
    #model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(128, 2, 2,activation='relu'))
    model.add(Conv2D(128, 2, 2,activation='relu'))
    #model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(256, 2, 2,activation='relu'))
    model.add(Conv2D(256, 2, 2,activation='relu'))
    model.add(Conv2D(256, 2, 2,activation='relu'))
    #model.add(MaxPooling2D((2,2), strides=(2,2)))

    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(2, activation='linear'))

    return model


def NN(weights_path=None):

    model = Sequential()
    model.add(Dense(300, activation='relu',input_shape=(225,)))
    model.add(Dense(300, activation='relu'))
    model.add(Dense(600, activation='relu'))
    model.add(Dense(600, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(4, activation='linear'))

    return model

model=VGG19()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 127, 127, 64)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 64)      16448     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 125, 125, 128)     32896     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 124, 124, 128)     65664     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 123, 123, 256)     131328    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 122, 122, 256)     262400    
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 121, 121, 256)    

/home/lab1_ysy/anaconda3/envs/lab1_ysy/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), activation="relu", input_shape=(128, 128,...)`
/home/lab1_ysy/anaconda3/envs/lab1_ysy/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), activation="relu")`
/home/lab1_ysy/anaconda3/envs/lab1_ysy/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (2, 2), activation="relu")`
/home/lab1_ysy/anaconda3/envs/lab1_ysy/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (2, 2), activation="relu")`
/home/lab1_ysy/anaconda3/envs/lab1_ysy/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (2, 2), activation="relu")`
/home/lab1

In [ ]:
#pyplot_model(model, to_file=save+str(count)+'_Architecture_'+name+'.png', show_shapes=True)

adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer= adam , loss='mse', metrics=['mae','accuracy'])
early_stopping = EarlyStopping(patience=10)
hist = model.fit(X_train_reshape, Y_train_scale, batch_size=36, epochs=1000,validation_split=0.1,callbacks=[early_stopping],verbose=1)

Train on 905 samples, validate on 101 samples
Epoch 1/1000


In [ ]:
score = model.evaluate(X_test_reshape,Y_test_scale, verbose=0)
np.savetxt(save+str(count)+'_Score_'+name+'.txt', score)

print(model.metrics_names)
print(score)

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'],label='train loss')
loss_ax.plot(hist.history['val_loss'],label='val_loss')
#loss_ax.plot(hist.history['mean_absolute_error'],label='train acc')
#loss_ax.plot(hist.history['val_mean_absolute_error'],label='val acc')
plt.savefig(save+str(count)+'_Train_curve_'+name+'.png', dpi=300)

In [ ]:
Y_hat_test = abs(model.predict(X_test_reshape,batch_size=32))
Y_hat_test_inverse = sc.inverse_transform(Y_hat_test)
Y_test_inverse = sc.inverse_transform(Y_test_scale)

record_pred = []

def fl(list):
    l = list
    a = format(l[0], "1.5f")
    b = format(l[1], "1.5f")
    list = [a,b]
    return list

for i in range(len(X_test)):
    Y_hat= Y_hat_test_inverse[i]
    print(i,'real: [freq_11,mass]',fl(Y_test_inverse[i]),'pred:[freq_11,mass]',fl(Y_hat),' ',sep='\n')
    Y_append = np.append(Y_test_inverse[i],Y_hat)
    record_pred.append(Y_append)  

np.savetxt(save+str(count)+'_Record pred_'+name+'.csv', record_pred,delimiter=',')
model.save_weights(save+str(count)+"_Weight_"+name+".h5")

record_mae = []
pred_shape = np.shape(record_pred)
for i in range(pred_shape[0]):
    record_mae.append(MAE(record_pred[i][0:1],record_pred[i][2:3]))

final_metric = np.mean(record_mae)

print('final mean_absolute_error:',format(final_metric,"1.4f"))